# Getting started with py4DSTEM

This notebook is a very simple introduction to py4DSTEM.  This notebook is intended to help you get comfortable with using the basic functonality of py4DSTEM, including concepts such as loading or saving data, or plotting images.

### Table of Contents

* [Part 0 - Import py4DSTEM, download a 4D-STEM dataset, load it.](#part_00)
* [Part 1 - Basic visualization](#part_01)
* [Part 2 - Basic visualization](#part_01)



### Acknowledgements

This notebook was made by the py4DSTEM team.


---
# Part 0 - Import py4DSTEM, download a 4D-STEM dataset.<a class="anchor" id="part_00"></a>


In [35]:
# Initialization - import the needed packages.
# For this tutorial, we will need py4DSTEM and numpy.
# numpy is the package we use to interact with vectors and arrays, and perform math operations.
import py4DSTEM
import numpy as np

You can download the tutorial dataset here: (51.8 megabytes)
* [10 x 10 probe positions 4D-STEM scan in dm3 format](https://drive.google.com/file/d/1B-xX3F65JcWzAg0v7f1aVwnawPIfb5_o/view?usp=sharing)

If you are running in **Binder** run the cell below, otherwise skip

In [36]:
# TODO - add binder link to file_getter
# %run /home/jovyan/file_getter.py --small4DSTEMscan

In [37]:
# Set the file paths for the input datasets.
# The location of these files may be different, depending on your operating system and browser download path.
# If you are running this tutorial on Binder, you do not need to change this path.
# file_path_input = '/home/jovyan/data/carbon_nanotube_data.h5'
file_path_input = 'C:/data/small4DSTEMscan_10x10.dm3'

In [38]:
# Load the dm3 file (Gatan digital micrograph format) whose path we just defined
datacube = py4DSTEM.io.read(file_path_input)

In [17]:
# What we just did was 'open' the dm3 file, but as a py4DSTEM DataCube.
# But, what exactly is a py4DSTEM datacube?  Let's try just running it as a python console input:
datacube

Well that wasn't very helpful if you don't already know what a DataCube is!  A py4DSTEM Datacube is a python class. Classes can have properties and methods. The object we defined is named 'datacube', but what it is, is a DataCube-type object. 

You yourself have a name, but we could perhaps say your class is 'human' (or so I assume).

In [26]:
# Let's look at some properties of our class, by putting them in square brackets (a python list)
[datacube.data, datacube.R_N, datacube.Q_Nx, datacube.Q_Ny]

[array([[[[23, 10,  0, ..., 17, 24,  5],
          [ 6,  0,  0, ..., 18,  6,  0],
          [ 0,  0, 26, ...,  5,  0,  0],
          ...,
          [ 0,  5,  0, ...,  5,  0,  9],
          [ 0,  0,  0, ...,  9,  0, 21],
          [27,  0, 11, ...,  0,  0, 17]]],
 
 
        [[[23, 12,  0, ..., 10,  1,  0],
          [ 1,  0,  0, ...,  0,  6,  0],
          [ 0,  0, 23, ...,  3,  0,  0],
          ...,
          [ 0,  3,  3, ...,  0,  0,  3],
          [ 7,  9,  0, ...,  2,  0, 23],
          [29,  0, 10, ...,  0,  0,  0]]],
 
 
        [[[20,  7, 12, ...,  0,  5,  9],
          [ 0,  0,  0, ..., 16,  5,  0],
          [ 0,  0, 13, ...,  3,  0,  0],
          ...,
          [ 0, 12,  1, ...,  3,  1, 17],
          [ 0, 18,  0, ...,  0,  0, 18],
          [38,  0,  0, ...,  0,  0, 13]]],
 
 
        ...,
 
 
        [[[21,  8, 11, ..., 10, 14,  3],
          [ 5,  0,  0, ..., 27,  4,  0],
          [ 1,  0, 25, ..., 11,  0,  0],
          ...,
          [ 0, 12, 11, ...,  6,  0,  9],
   

The first object in our list is the dataset itself - it's quite large, so python (smartly) displays only a little bit of it.

We can see some other properies below the dataset:
* datacube.R_N  = 100
* datacube.Q_Nx = 512
* datacube.Q_Ny = 512

You may have already guessed based on the names, but these are the size of the (3D) array 'data' - The diffraction space dimensions (labeled with Q) are 512 x 512 pixels, and we have recorded 100 probe positions. This shows one of the limitations of the dm3 format - it can only store a 3D array, not a 4D array. However, from the file name we see this scan recorded 10 x 10 probe positions.

In [33]:
# We will try manually reshaping the dataset to have the correct shape, using a DataCube method 'set_scan_shape'
datacube.set_scan_shape(100,100)

Can't reshape 100 scan positions into a 100x100 array.


In [34]:
# Oops!  A typo - this demonstrates the method is smart enough to know when our new shape is impossible. 
# Setting the correct shape:
datacube.set_scan_shape(10,10)

In [2]:
import os


In [3]:
os.getcwd()

'C:\\Users\\Alex\\Documents\\Git-Repositaries\\py4DSTEM_tutorials'